##### Baseline:      
    1) Dimensionality reduction to capture principal modes of app-usage
    2) Cluster activities based on clusters in the principal dimensions

### Get cleaned data

In [1]:
from importlib import reload
from utils import *
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import pickle
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from gmeans_mahalonobis_2 import *
from sklearn.mixture import GaussianMixture
# plotly.tools.set_credentials_file(username='AbdelWahabTurkmani', api_key='zTY632QGUJbqSVFms8bQ')
# plotly.tools.set_credentials_file(username='asturkmani', api_key='bR0Ez8evmzcHY6m4XG7B')
plotly.tools.set_credentials_file(username='asturkmanics', api_key='GaUFc6cWCQVW0eCz6K75')
%load_ext autoreload
%autoreload 2

In [23]:
time_percentage = 0.9
explained_variance = 0.9
# df = pd.read_csv("data/rescuetime_data-ac-min.csv")
# data_pd = Clean_DF(df)
# data_pd.clean_data(time_percentage=time_percentage)
# data_pd.clean_df = data_pd.clean_df.reset_index()
# data_pd.get_pca(explained_variance=explained_variance)
# data_pd.get_day_time()
# obj0, obj1, obj2 are created here...

# Saving the objects:
# with open('tsne_results_pca.pickle', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(tsne_results_pca, f)

# Getting back the objects:
with open('data_pd.pickle', 'rb') as f:  # Python 3: open(..., 'rb')
    data_pd = pickle.load(f)


In [3]:
print("Dataset size:", data_pd.clean_df.shape,'\n')
print("Number of apps that consume", time_percentage*100, "% of all users time: ",len(data_pd.popular_apps), '\n')
print("Cleaned dataset columns:",'\n', data_pd.clean_df.columns.values, '\n')
print("Number of components that explain", explained_variance*100,"% of the data: ",data_pd.pca_data.shape[1], '\n')

Dataset size: (16704, 9) 

Number of apps that consume 90.0 % of all users time:  99 

Cleaned dataset columns: 
 ['Date' 'Time Spent (seconds)' 'Activity' 'Category' 'Productivity'
 'Activity Vector' 'Productivity Score' 'Day' 'Time'] 

Number of components that explain 90.0 % of the data:  30 



### Compute Principal Components and visualize top-3 modes

### Helps us visualize productive vs unproductive time as specified by the users selection

In [4]:
c = data_pd.clean_df['Productivity Score']
x = data_pd.pca_data[:,0]
y = data_pd.pca_data[:,1]
z = data_pd.pca_data[:,2]
t = data_pd.clean_df['Activity']
t = data_pd.clean_df['Activity'].tolist()
t = ['-'.join(x) for x in t]

trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='RdYlGn',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PCA-012 Visualization')

In [6]:
c = data_pd.clean_df['Productivity Score']
x = data_pd.pca_data[:,2]
y = data_pd.pca_data[:,3]
z = data_pd.pca_data[:,4]
t = data_pd.clean_df['Activity']
t = data_pd.clean_df['Activity'].tolist()
t = ['-'.join(x) for x in t]

trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='RdYlGn',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PCA-123 Visualization')

### Visualize entire PCA dimensional space using t-SNE

In [20]:
# tsne_pca = TSNE(n_components=3, verbose=0, perplexity=30, n_iter=5000)
# tsne_results_pca = tsne_pca.fit_transform(data_pd.pca_data)

with open('tsne_results_pca.pickle', 'rb') as f:  # Python 3: open(..., 'rb')
    tsne_results_pca = pickle.load(f)

In [ ]:
c = data_pd.clean_df['Productivity Score']
x = tsne_results_pca[:,0]
y = tsne_results_pca[:,1]
z = tsne_results_pca[:,2]
# t = data_pd.clean_df['Activity']
# t = data_pd.clean_df['Activity'].tolist()
# t = ['-'.join(x) for x in t]

trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='RdYlGn',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='t-SNE Productivity Score')

## Clustering data in PCA dimensions

### Clustering data using the elbow method

    1) Run Kmeans for N iterations recording the inertia for each
    2) At each iteration save the error
    3) When viewing the errors, pick the iteration that is at the elbow of the error curve


In [7]:
inertia = np.zeros(29)
for i in range(2,30):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data_pd.pca_data)
    inertia[i-2] = kmeans.inertia_

In [8]:
trace = go.Scatter(x=[i for i in range(0,25)], y=inertia, mode='markers')
data= [trace]
py.iplot(data, filename='K-Means inerta')

# Visualize data projected onto top-3 PCA components with G-Means labels

In [9]:
gmeans_unscaled = GMeans(critical_value=1.8692, verbose=0, recalculate_points=False)
gmeans_unscaled.fit(data_pd.activity_vector)
print("G-Means with unscaled data finds: ", len(gmeans_unscaled.cluster_centers_)," clusters")

/Users/asturkmani/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning:

Explicit initial center position passed: performing only one init in k-means instead of n_init=10



G-Means with unscaled data finds:  42  clusters


In [10]:
c = gmeans_unscaled.labels_
x = data_pd.pca_data[:,2]
y = data_pd.pca_data[:,3]
z = data_pd.pca_data[:,4]
t = data_pd.clean_df['Activity']
t = data_pd.clean_df['Activity'].tolist()
t = ['-'.join(x) for x in t]
for i in range(0,len(t)):
    t[i] = str(c[i]) + '---' + t[i]
    
trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='Viridis',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PCA-234 Visualization with G-Means and unscaled data')

In [11]:
gmeans_scaled = GMeans(critical_value=1.8692, verbose=0, recalculate_points=False)
gmeans_scaled.fit(scale(data_pd.activity_vector))
print("G-Means with scaled data finds: ", len(gmeans_scaled.cluster_centers_)," clusters")

/Users/asturkmani/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning:

Explicit initial center position passed: performing only one init in k-means instead of n_init=10



G-Means with scaled data finds:  34  clusters


In [12]:
c = gmeans_scaled.labels_
x = data_pd.pca_data[:,2]
y = data_pd.pca_data[:,3]
z = data_pd.pca_data[:,4]
t = data_pd.clean_df['Activity']
t = data_pd.clean_df['Activity'].tolist()
t = ['-'.join(x) for x in t]
for i in range(0,len(t)):
    t[i] = str(c[i]) + '---' + t[i]
    
trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='Viridis',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PCA-234 Visualization with G-Means and scaled data')

In [89]:
gmeanspp_unscaled = GMeans(critical_value=1.8692, verbose=0, recalculate_points=True)
gmeanspp_unscaled.fit(data_pd.activity_vector)
print("G-Means++ with unscaled data finds: ", len(gmeanspp_unscaled.cluster_centers_)," clusters")

/Users/asturkmani/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning:

Explicit initial center position passed: performing only one init in k-means instead of n_init=10

/Users/asturkmani/Desktop/Code/gmeans_mahalonobis_2.py:41: RuntimeWarning:

invalid value encountered in less



G-Means++ with unscaled data finds:  20  clusters


In [90]:
c = gmeanspp_unscaled.labels_
x = data_pd.pca_data[:,2]
y = data_pd.pca_data[:,3]
z = data_pd.pca_data[:,4]
t = data_pd.clean_df['Activity']
t = data_pd.clean_df['Activity'].tolist()
t = ['-'.join(x) for x in t]
for i in range(0,len(t)):
    t[i] = str(c[i]) + '---' + t[i]
    
trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='RdYlGn',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PCA-234 Visualization with G-Means++ unscaled data')

In [83]:
gmeanspp_scaled = GMeans(critical_value=1.8692, verbose=0, recalculate_points=True)
gmeanspp_scaled.fit(scale(data_pd.activity_vector))
print("G-Means++ with scaled data finds: ", len(gmeanspp_scaled.cluster_centers_)," clusters")

/Users/asturkmani/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning:

Explicit initial center position passed: performing only one init in k-means instead of n_init=10



G-Means++ with scaled data finds:  2  clusters


In [84]:
unique, counts = np.unique(gmeanspp_scaled.labels_, return_counts=True)
dict(zip(unique, counts))

{0: 7032, 1: 9672}

In [85]:
c = gmeanspp_scaled.labels_
x = data_pd.pca_data[:,2]
y = data_pd.pca_data[:,3]
z = data_pd.pca_data[:,4]
t = data_pd.clean_df['Activity']
t = data_pd.clean_df['Activity'].tolist()
t = ['-'.join(x) for x in t]
for i in range(0,len(t)):
    t[i] = str(c[i]) + '---' + t[i]
    
trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c,colorscale='RdYlGn',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PCA-234 Visualization with G-Means++ scaled data')

### Visualize original data with k-Means clusters computed in PCA dimensions

In [86]:
c = gmeanspp_scaled.labels_
x = tsne_results_pca[:,0]
y = tsne_results_pca[:,1]
z = tsne_results_pca[:,2]
t = data_pd.clean_df['Activity']
t = data_pd.clean_df['Activity'].tolist()
t = ['-'.join(x) for x in t]
for i in range(0,len(t)):
    t[i] = str(c[i]) + '---' + t[i]

trace1 = go.Scatter3d(x=x,y=y,z=z,text=t, mode='markers',marker=dict(size=12,color=c, colorscale = 'RdYlGn', opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='t-SNE Visualization with G-Means++ scaled data')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~asturkmanics/0 or inside your plot.ly account where it is named 't-SNE Visualization with G-Means++ scaled data'
